### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
#Gets the number of players with unique screen names
total_players = purchase_data['SN'].nunique()

#puts the value from above into dataframe to make it more readable
player_total = pd.DataFrame(data={'Total Players': [total_players]})
player_total

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#getting number of unique items by filtering through their ID
unique_items = purchase_data['Item ID'].nunique()

#stores number for average_price from price column in purchase_data
average_price = purchase_data['Price'].mean()

#counts number of rows in purchases_data which equals total purchases
num_purchases = len(purchase_data.index)

#adds every value in price column to give total revenue
total_revenue = purchase_data['Price'].sum()

#puts the data into each appropriate column and formats the data to make it readable
neat_summary = pd.DataFrame(data={'Number of Unique Items': [unique_items], 'Average Price': [average_price], 'Number of Purchases': [num_purchases], 'Total Revenue':[total_revenue]})
neat_summary['Average Price'] = neat_summary['Average Price'].map('${:,.2f}'.format)
neat_summary['Total Revenue'] = neat_summary['Total Revenue'].map('${:,.2f}'.format)
neat_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#makes dataframe using only unique names
unique_users = purchase_data.drop_duplicates(subset='SN')

#appends count of each gender to dict using unique users
gender_count = unique_users['Gender'].value_counts().to_dict()

#append dictionary to dataframe
gender_count = pd.DataFrame.from_dict(gender_count, orient='index', columns=['Total Count'])

#making new column 'Percent' based on data in first column
gender_count = gender_count.assign(Percent=lambda x: x['Total Count'] / total_players * 100)

#cleaning up the dataframe
gender_count = gender_count.rename(index=str, columns={"Percent":"Percentage of Players"})
gender_count['Percentage of Players'] = gender_count['Percentage of Players'].map('{:,.2f}%'.format)
gender_count

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#initilizing a dataframe for gender data summary
gender_summary = purchase_data['Gender'].value_counts().to_dict()
gender_summary = pd.DataFrame.from_dict(gender_summary, orient='index', columns=['Purchase Count'])


#sorting data by females and getting the average prices based on total gender data and unique gender data
fem_sum = purchase_data[purchase_data.Gender == 'Female']
#adds up everything in the price column
fem_total = fem_sum['Price'].sum()
#average price spent by females
av_femprice = fem_total / fem_sum['Purchase ID'].nunique()
#gets count for unique female users
unique_fems = fem_sum.drop_duplicates(subset='SN')
#counts number of unique female users
num_fems = unique_fems['Gender'].value_counts()
#price per female is equal to total female spending(fem_total) divided by number of unique female users
price_perfem = fem_total / num_fems


#same steps as above, except gender is sorting out the males
mal_sum = purchase_data[purchase_data.Gender == 'Male']
mal_total = mal_sum['Price'].sum()
av_malprice = mal_total / mal_sum['Purchase ID'].nunique()
unique_mal = mal_sum.drop_duplicates(subset='SN')
num_mal = unique_mal['Gender'].value_counts()
price_permal = mal_total / num_mal


#same steps again for other
oth_sum = purchase_data[purchase_data.Gender == 'Other / Non-Disclosed']
oth_total = oth_sum['Price'].sum()
av_othprice = oth_total / oth_sum['Purchase ID'].nunique()
unique_oth = oth_sum.drop_duplicates(subset='SN')
num_oth = unique_oth['Gender'].value_counts()
price_peroth = oth_total / num_oth


#adds columns with their respective values calculated above
gender_summary.loc[:,'Average Purchase Price'] = pd.Series([av_malprice, av_femprice, av_othprice], index=gender_summary.index)
gender_summary.loc[:,'Total Purchase Value'] = pd.Series([mal_total, fem_total, oth_total], index=gender_summary.index)
#This line is causing a weird error in my column and causes my last line of formatting to fail when not voided out
gender_summary.loc[:,'Avg Total per Person'] = pd.Series([price_permal, price_perfem, price_peroth], index=gender_summary.index)

#formats numbers
gender_summary['Average Purchase Price'] = gender_summary['Average Purchase Price'].map('${:,.2f}'.format)
gender_summary['Total Purchase Value'] = gender_summary['Total Purchase Value'].map('${:,.2f}'.format)
#gender_summary['Avg Total per Person'] = gender_summary['Avg Total per Person'].map('${:,.2f}'.format)


gender_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total per Person
Male,652,$3.02,"$1,967.64","Male 4.065372 Name: Gender, dtype: float64"
Female,113,$3.20,$361.94,"Female 4.468395 Name: Gender, dtype: float64"
Other / Non-Disclosed,15,$3.35,$50.19,Other / Non-Disclosed 4.562727 Name: Gender...


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [32]:
#creating bins and labels for binning
bins = [0, 10, 15, 20, 25, 30, 35, 40, 200]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#binning data and adding columns
age_data = purchase_data['Age']
age_df = pd.DataFrame(data={'Age': age_data})
age_df['Total Count'] =pd.cut(age_data, bins, labels=labels).value_counts()
age_df = age_df.assign(Percent=lambda x: x['Total Count'] / total_players * 100)
age_df



,Age,Total Count,Percent
0,20,NaN,NaN
1,40,NaN,NaN
2,24,NaN,NaN
3,24,NaN,NaN
4,23,NaN,NaN
5,22,NaN,NaN
6,36,NaN,NaN
7,20,NaN,NaN
8,22,NaN,NaN
9,35,NaN,NaN


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [37]:
#setting up data to be sorted using groupby function
spender_summary = purchase_data['SN'].value_counts().to_dict()
spender_summary = pd.DataFrame.from_dict(spender_summary, orient='index', columns=['Purchase Count'])
spenders_summary = purchase_data.groupby('SN').agg({'Price': ['sum','count','mean']})

spenders_summary



Price                
                  sum count      mean
SN                                   
Adairialis76     2.28     1  2.280000
Adastirin33      4.48     1  4.480000
Aeda94           4.91     1  4.910000
Aela59           4.32     1  4.320000
Aelaria33        1.79     1  1.790000
Aelastirin39     7.29     2  3.645000
Aelidru27        1.09     1  1.090000
Aelin32          8.98     3  2.993333
Aelly27          6.79     2  3.395000
Aellynun67       3.74     1  3.740000
Aellyria80       3.08     1  3.080000
Aelollo59        5.63     2  2.815000
Aenarap34        2.96     1  2.960000
Aeral43          4.40     1  4.400000
Aeral68          4.00     1  4.000000
Aeral97          4.80     1  4.800000
Aeralria27       4.09     1  4.090000
Aeralstical35    2.96     1  2.960000
Aeri84           1.61     1  1.610000
Aerillorin70     3.33     1  3.330000
Aerithllora36    8.64     2  4.320000
Aerithnucal56    4.40     1  4.400000
Aerithnuphos61   4.91     1  4.910000
Aerithriaphos45  1.56     1  1.560000
Aerithriaphos46  2.18     1  2.180000
Aesri53          1.76     1  1.760000
Aesty53          3.91     2  1.955000
Aestysu37        5.38     2  2.690000
Aesur96          2.21     1  2.210000
Aesurstilis64    1.03     1  1.030000
...               ...   ...       ...
Undosia27        3.77     1  3.770000
Undosian34       8.30     2  4.150000
Undotesta33      4.74     1  4.740000
Wailin72         2.73     1  2.730000
Yadacal26        6.54     2  3.270000
Yadaisuir65      4.09     1  4.090000
Yadam35          2.48     1  2.480000
Yadanu52         2.38     1  2.380000
Yadaphos40       5.35     2  2.675000
Yalae81          6.69     2  3.345000
Yalaeria91       1.06     1  1.060000
Yaliru88         3.03     1  3.030000
Yalo85           3.65     1  3.650000
Yalostiphos68    4.40     1  4.400000
Yana46           3.55     1  3.550000
Yarithllodeu72   3.75     1  3.750000
Yarithrgue83     3.45     1  3.450000
Yarithrin84      4.14     1  4.140000
Yarithsurgue62   4.94     1  4.940000
Yarolwen77       4.43     1  4.430000
Yasrisu92        3.44     1  3.440000
Yastyriaphos75   4.75     1  4.750000
Yasur35          4.48     1  4.480000
Yasur85          1.10     1  1.100000
Yathecal72       3.46     1  3.460000
Yathecal82       6.22     3  2.073333
Yathedeu43       6.02     2  3.010000
Yoishirrala98    4.58     1  4.580000
Zhisrisu83       7.89     2  3.945000
Zontibe81        8.03     3  2.676667

[576 rows x 3 columns]

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
